In [ ]:
# imports
import numpy as np
import krakenex as k
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
import datetime
from pathlib import Path
import sys
from datetime import datetime, timedelta
from matplotlib import dates 
from sqlalchemy import create_engine, MetaData, Table, Column, DateTime, Float, String,Integer

dbInstance = 'sqlite:///krakenTeader.db'
dbBalanceHistoryTable = 'BalanceHistory'
dbTradesHistoryTable = 'TradesHistory'
engine = create_engine(dbInstance)


In [ ]:


#preparar datos para plotear
BalanceHistory = pd.read_sql(dbBalanceHistoryTable, con=engine)
BalanceHistory = BalanceHistory.set_index(pd.DatetimeIndex(BalanceHistory['Time']))
BalanceHistory.drop('Time', axis=1,inplace=True)

#filtrar periodo de interes
windowTime = 24 * 7
EndSampleTime = datetime(2017,10,18,0,0)
plotmin = True
HoursInterval = 1
if windowTime > 6: plotmin = False
if windowTime > 40: HoursInterval = 2
if windowTime > 60: HoursInterval = 3
if windowTime > 80: HoursInterval = 4

verUltimo=False

if verUltimo:
    startSampleTime = datetime.now() + timedelta(hours=windowTime*-1)
    BalanceHistory = BalanceHistory[BalanceHistory.index > startSampleTime]
else:
    startSampleTime = EndSampleTime + timedelta(hours=windowTime*-1)
    BalanceHistory = BalanceHistory[(BalanceHistory.index >= startSampleTime) & (BalanceHistory.index <= EndSampleTime)]

#combertir el balance a un porcentaje entre -100% y 100%
BalanceHistory['balanceRatio'] = (BalanceHistory['balanceRatio'] - 0.5 )*2*100 

BalanceHistory['volbuy'] = BalanceHistory['volbuy'] / BalanceHistory['close']
BalanceHistory['volsell'] = BalanceHistory['volsell'] / BalanceHistory['close']
BalanceHistory['unbalance'] = BalanceHistory['unbalance'] / BalanceHistory['close']

#calcular %CH, %CH Acum , SMA %CH Acum
BalanceHistory['change'] = BalanceHistory['close'].pct_change(periods=1)*100
#BalanceHistory['cahnge2'] = BalanceHistory['close'] / BalanceHistory['close'].shift(1) -1 #equibalente 
BalanceHistory['cum_change'] = BalanceHistory['change'].cumsum()
BalanceHistory['SMA03_cum_change'] = BalanceHistory['cum_change'].rolling(5).mean()
BalanceHistory['SMA12_cum_change'] = BalanceHistory['cum_change'].rolling(12).mean()

#calcular sma de 3 periodos (suavisar el grafico)
BalanceHistory['EWM_unbalance'] =  BalanceHistory["unbalance"].ewm(span=3).mean()

#calcular la porsiocn negativa (para mostrar en rojo en el grafico)
BalanceHistory['EWM_unbalance_N'] = BalanceHistory['EWM_unbalance']
neg = BalanceHistory['EWM_unbalance_N']
neg[neg >= 0] = np.nan
BalanceHistory['EWM_unbalance_N'] = neg


comCompra = 0.29  
comVenta = 0.19 
spreadEntrada = 0.2 
spreadSalida = 0.2
deltaBaseCH = comCompra + comVenta + spreadEntrada + spreadSalida 

In [ ]:
def upTrend (bh, chOpen = 0.5, mbOpen = 0.5, deltaCHObjetivo = 5, chClose = 1.2, deltaTotalLoseCH = 1, waitPeriods = 60, waitFactor = 9, waitPeriodsOutBase = 30, waitFactorOutBase = 5 ):
             
    comCompra = 0.29  
    comVenta = 0.19 
    spreadEntrada = 0.2 
    spreadSalida = 0.2
    deltaBaseCH = comCompra + comVenta + spreadEntrada + spreadSalida 

    deltaTargetCH = deltaBaseCH + deltaCHObjetivo
    deltaStopLose = deltaBaseCH - chClose

    inBase = False
    
    s = bh['unbalance']
    sma03 = bh['SMA03_cum_change']
    sma12 = bh['SMA12_cum_change']
    ch = bh['change']
    
    #volb = tc['volb']
    
    openPos = 0
    isOpen = False
    inBase = False
    
    sOpenCond = ""
    sCloseCond = ""
        
    #valores de cada operacion 
    openingCH = 0.0
    baseCH = 0.0
    targetCH = 0.0
    stopLoseCH = 0.0
    TotalLoseCH = 0.0
    closingCH = 0.0
    deltaCH = 0.0
    idTrade = 0
    openTime = datetime.now()
    closeTime =openTime

    cols = ['id','openTime','closeTime','tradeDescription','openingCH','baseCH','targetCH','stopLoseCH','TotalLoseCH','closingCH','deltaCH'] 
    myTrades = pd.DataFrame(columns=cols)
   
    for i in range(len(s)):
        if i > 12:
            if (isOpen == False):
                if (s[i] > 0) & (s[i-1] < 0):
                    if (sma03[i] > sma03[i-1]+ chOpen):# & (volb[i] > 50):
                        isOpen = True
                        sOpenCond = "MB UP 1"                
                elif (s[i] > 0) & (s[i] > mbOpen): # & (volb[i] > 50):
                    isOpen = True
                    sOpenCond = "MB UP 2"                
                elif (s[i] < 0) & (s[i-1] > s[i-2] + mbOpen*2): # & (volb[i] > 50):
                    isOpen = True
                    sOpenCond = "MB UP 3"                
                elif ((sma03[i-1] < sma12[i-1]) & (sma03[i] > sma12[i])): 
                    if (sma03[i] > sma03[i-1]+ chOpen): #  & (volb[i] > 50):
                        isOpen = True
                        sOpenCond = "SMA03 UP"
                elif (ch[i] > ch[i-1] + chOpen): 
                    if (sma03[i] > sma03[i-1]+ chOpen): # & (volb[i] > 50):
                        isOpen = True
                        sOpenCond = "CH UP"
                
                if isOpen == True:
                    openPos = i
                    idTrade = idTrade + 1
                    sDesc = '({0}) ─► {1}'.format(idTrade, sOpenCond)
                    inBase = False
                    openTime = s.index[i]
                    openingCH = bh.cum_change[i]
                    baseCH = openingCH + deltaBaseCH
                    targetCH = openingCH + deltaTargetCH
                    stopLoseCH = openingCH + deltaStopLose
                    TotalLoseCH = openingCH - deltaTotalLoseCH
                    
            
            if (isOpen == True):
                if bh.cum_change[i] >= baseCH: #nivel de perdidas operacionales superado
                    inBase = True
                
                if inBase == True: 
                    if bh.cum_change[i] >= targetCH: #objetivo cumplido
                        if s[i] < mbOpen: #dejar correr las ganancias
                            isOpen = False
                            sCloseCond = "TARGET ACCOMPLISHED AT {0} %CH".format(round(deltaTargetCH,2))
                        
                    if bh.cum_change[i] <= (stopLoseCH): #cierre por stop lose
                        isOpen = False
                        sCloseCond = "STOP LOSE AT {0} %CH".format(round(deltaStopLose,2))
                    elif (i >= (openPos + waitPeriods*waitFactor)): #cierre por tiempo trascurrido sin logar objetivos
                        isOpen = False
                        sCloseCond = "elapsed {0} times without goals | IN BASE".format(waitPeriods*waitFactor)
                        '''
                        if ((sma03[i-1] > sma12[i-1]) & (sma03[i] <= sma12[i])): #cierre por sma corta sma larga
                            isOpen = False
                            sCloseCond = "SMA03 DOWN"
                        '''
                else: #nivel de perdidas operacionales no se logro 
                    if (s[i] <= 0) &(bh.cum_change[i] < TotalLoseCH): #cierre por perdida total
                        isOpen = False
                        sCloseCond = "TOTAL LOSE"
                    elif (i >= (openPos + waitPeriodsOutBase*waitFactorOutBase)): #cierre por tiempo trascurrido sin logar objetivos
                        isOpen = False
                        sCloseCond = "elapsed {0} times without goals | OUT BASE".format(waitPeriods*waitFactor)
                        
                if isOpen == False:
                    #resC.append(s.index[i])
                    sDesc = sDesc + ' | {0}'.format(sCloseCond)
                    #resDesc.append(sDesc)
                    inBase = False
                    
                    #v2
                    closeTime = s.index[i]
                    closingCH = bh.cum_change[i]
                    if openingCH >= closingCH:
                        deltaCH = closingCH - openingCH
                    else:
                        deltaCH = closingCH - openingCH
                    newTrade = [idTrade,openTime,closeTime,sDesc,openingCH,baseCH,targetCH,stopLoseCH, TotalLoseCH,closingCH,deltaCH]
                    myTrades.loc[len(myTrades)] = newTrade   
                    #fin v2
                    
            if (i == len(s)) & (isOpen == True): #ultima posision y algun trade sigue abierto
                sDesc = sDesc + ' | IN PROGRESS'
                closeTime = s.index[i]
                closingCH = bh.cum_change[i]
                if openingCH >= closingCH:
                    deltaCH = closingCH - openingCH
                else:
                    deltaCH = closingCH - openingCH
                newTrade = [idTrade,openTime,closeTime,sDesc,openingCH,baseCH,targetCH,stopLoseCH, TotalLoseCH,closingCH,deltaCH]
                myTrades.loc[len(myTrades)] = newTrade
    
    print('Total Trades: {0}'.format(len(myTrades)))
    myTrades.set_index('id',inplace=True)
    return myTrades

In [ ]:
Configcols = ['id','totalWin','totalCom','finalBalance','chOpen','mbOpen','deltaCHObjetivo','chClose','deltaTotalLoseCH', 'waitPeriods','waitFactor', 'waitPeriodsOutBase','waitFactorOutBase'] 
allConfigs = pd.DataFrame(columns=Configcols)
    
def TestTradeConfigs():
    #parametros de operaciones
    # Entrada
    chOpen = 0.3            #cambio que se tiene que producir para abrir una posicion 
    mbOpen = 0.3            #desbalance minimo  POs para abrir un aposision
    deltaCHObjetivo = 2.5  #%CH que espero alcanzar
    
    # Salida con perdida
    chClose = 1.2           #%CH que admito como riego cuando ya se cubrieron los gastos OPeracionales (Riesgo medio)
    deltaTotalLoseCH = 1  #%CH por debajo de la apertura para cerrar ocn perdida total (RIESGO ALTO)
    
    # Espera 
    waitPeriods = 60        #cantidad de unidades (las mismas que el BalanceHistory) que espero para cerrrar sin objetivo
    waitFactor = 10          #cantidad de waitPeriods que espero (segun factor de escala)
    
    # Espera (RIESGO ALTO)
    waitPeriodsOutBase = 30 #cantidad de unidades (las mismas que el BalanceHistory) que espero para cerrrar si no se llaga a base
    waitFactorOutBase = 10  #cantidad de waitPeriods que espero (segun factor de escala)
    
    Tradescols = ['id','openTime','closeTime','tradeDescription','openingCH','baseCH','targetCH','stopLoseCH','TotalLoseCH','closingCH','deltaCH'] 
    myTrades = pd.DataFrame(columns=Tradescols)
    
    Contador = 0
    for chOpenVal in np.arange(0.4,1,0.1):
        for mbOpenVal in  np.arange(0.5,1,0.1):
            for deltaCHObjetivoVal in  np.arange(3,8,0.2):
                #for chCloseVal in  np.arange(0.1,3,0.5):
                    
                #for deltaTotalLoseCHVal in  np.arange(0.1,3,0.2):
                #    for waitFactorVal in range(1,24,1):
                #        for waitFactorOutBaseVal in range(1,12,1):

                myTrades = upTrend(BalanceHistory, chOpenVal, mbOpenVal,deltaCHObjetivoVal, chClose, deltaTotalLoseCH, waitPeriods, waitFactor, waitPeriodsOutBase, waitFactorOutBase)

                totalCom = len(myTrades) * deltaBaseCH
                totalWin = myTrades.deltaCH.sum()
                finalBalance = totalWin - totalCom
                newConfig = [Contador, totalWin,totalCom,finalBalance,chOpenVal, mbOpenVal, deltaCHObjetivoVal, chClose, deltaTotalLoseCH, waitPeriods, waitFactor, waitPeriodsOutBase, waitFactorOutBase]
                allConfigs.loc[Contador] = newConfig

                print("Contador {0}, totalWin {1},totalCom {2},finalBalance {3},chOpenVal {4}, mbOpenVal {5}, deltaCHObjetivoVal {6}, chCloseVal {7}, deltaTotalLoseCH {8}, waitPeriods {9}, waitFactor {10}, waitPeriodsOutBase {11}, waitFactorOutBase {12}".format(Contador, totalWin,totalCom,finalBalance,chOpenVal, mbOpenVal, deltaCHObjetivoVal, chClose, deltaTotalLoseCH, waitPeriods, waitFactor, waitPeriodsOutBase, waitFactorOutBase))
                Contador += 1


In [ ]:
TestTradeConfigs()

In [13]:
allConfigs = pd.read_csv('AllConfigs.csv')

In [14]:
allConfigs.sort_values(['finalBalance','totalWin','totalCom'], ascending=[False,False, False], axis=0, inplace=True)
allConfigs.reset_index(inplace=True)
allConfigs.drop('index',inplace=True, axis=1)
allConfigs.head(100)

,Unnamed: 0,id,totalWin,totalCom,finalBalance,chOpen,mbOpen,deltaCHObjetivo,chClose,deltaTotalLoseCH,waitPeriods,waitFactor,waitPeriodsOutBase,waitFactorOutBase
0,0,11988.0,16.651607,4.4,12.251607,0.4,0.5,4.9,0.7,1.0,60.0,10.0,30.0,10.0
1,1,11989.0,16.651607,4.4,12.251607,0.4,0.5,4.9,0.9,1.0,60.0,10.0,30.0,10.0
2,2,11990.0,16.651607,4.4,12.251607,0.4,0.5,4.9,1.1,1.0,60.0,10.0,30.0,10.0
3,3,11991.0,16.651607,4.4,12.251607,0.4,0.5,4.9,1.3,1.0,60.0,10.0,30.0,10.0
4,4,11992.0,16.651607,4.4,12.251607,0.4,0.5,4.9,1.5,1.0,60.0,10.0,30.0,10.0
5,5,11993.0,16.651607,4.4,12.251607,0.4,0.5,4.9,1.7,1.0,60.0,10.0,30.0,10.0
6,6,11994.0,16.651607,4.4,12.251607,0.4,0.5,4.9,1.9,1.0,60.0,10.0,30.0,10.0
7,7,11995.0,16.651607,4.4,12.251607,0.4,0.5,4.9,2.1,1.0,60.0,10.0,30.0,10.0
8,8,11996.0,16.651607,4.4,12.251607,0.4,0.5,4.9,2.3,1.0,60.0,10.0,30.0,10.0
9,9,11997.0,16.651607,4.4,12.251607,0.4,0.5,4.9,2.5,1.0,60.0,10.0,30.0,10.0


In [ ]:
allConfigs.to_csv('AllConfigs.csv')

In [18]:
allConfigs[(allConfigs['mbOpen'] >= 0.6) & (allConfigs['chOpen'] >= 0.4)].head(100).sort_values(['finalBalance'], ascending=[False], axis=0)

,Unnamed: 0,id,totalWin,totalCom,finalBalance,chOpen,mbOpen,deltaCHObjetivo,chClose,deltaTotalLoseCH,waitPeriods,waitFactor,waitPeriodsOutBase,waitFactorOutBase
216,216,12738.0,14.178399,2.64,11.538399,0.4,0.7,4.9,0.7,1.0,60.0,10.0,30.0,10.0
253,253,22864.0,14.178399,2.64,11.538399,0.7,0.7,4.9,0.9,1.0,60.0,10.0,30.0,10.0
269,269,26243.0,14.178399,2.64,11.538399,0.8,0.7,4.9,1.7,1.0,60.0,10.0,30.0,10.0
268,268,26242.0,14.178399,2.64,11.538399,0.8,0.7,4.9,1.5,1.0,60.0,10.0,30.0,10.0
267,267,26241.0,14.178399,2.64,11.538399,0.8,0.7,4.9,1.3,1.0,60.0,10.0,30.0,10.0
217,217,12739.0,14.178399,2.64,11.538399,0.4,0.7,4.9,0.9,1.0,60.0,10.0,30.0,10.0
265,265,26239.0,14.178399,2.64,11.538399,0.8,0.7,4.9,0.9,1.0,60.0,10.0,30.0,10.0
264,264,26238.0,14.178399,2.64,11.538399,0.8,0.7,4.9,0.7,1.0,60.0,10.0,30.0,10.0
263,263,22874.0,14.178399,2.64,11.538399,0.7,0.7,4.9,2.9,1.0,60.0,10.0,30.0,10.0
262,262,22873.0,14.178399,2.64,11.538399,0.7,0.7,4.9,2.7,1.0,60.0,10.0,30.0,10.0
